In [1]:
#SET UP:

In [2]:
print("HELLO")

HELLO


In [3]:
print("he")

he


In [5]:
from pathlib import Path
import openmeteo_requests
from dotenv import load_dotenv
import datetime
import hopsworks
import openmeteo_requests

HOPSWORKS_HOST="c.app.hopsworks.ai"
#May need https://
HOPSWORKS_PROJECT="AirQualityKTHLab1"
city = "stockholm"
pollen_types = ["alder_pollen","birch_pollen","grass_pollen" ,"mugwort_pollen" ,"olive_pollen" ,"ragweed_pollen"]

today = datetime.date.today()

root_dir = Path.cwd()
print(root_dir)


load_dotenv()

project = hopsworks.login(host=HOPSWORKS_HOST, project=HOPSWORKS_PROJECT)


/home/viktor/Pollen-Predictor/notebooks
2025-12-31 00:39:15,094 INFO: Initializing external client
2025-12-31 00:39:15,095 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-12-31 00:39:16,114 WARNING: UserWarning: The installed hopsworks client version 4.6.0 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-12-31 00:39:17,004 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286344


In [ ]:
def fetch_historical_wether(latitude, longitude, date_start, data_end):





In [ ]:
def fetch_wether_forecast(latitude, longitude, forecast_days = 9):

    